# Dynamic Contrast Enhanced Image Reconstruction with BART

In [1]:
export PATH=${TOOLBOX_PATH}:${PATH}
export DEBUG_LEVEL=1

In this demo we will investigate prospectively under-sampled dynamic contrast enhanced (DCE) MRI data provided by [Tao
Zhang](http://web.stanford.edu/~tzhang08/). 

Unzip the data and it will be placed under `data/`.

In [7]:
unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/ksp.cfl            
  inflating: data/ksp.hdr            
  inflating: data/maps.cfl           
  inflating: data/maps.hdr           
  inflating: data/weights.cfl        
  inflating: data/weights.hdr        


The data consist of a single slice from under-sampled multi-channel, Cartesian k-space binned into temporal phases, as well as
ESPIRiT sensitivity maps and sampling weights derived
from navigator data. 

We will first look at the datasets. BART provides a simple multi-dimensional viewer `bartview.py` in python. We also have a more sophisticated viewer `view` in development available at:
    https://github.com/mrirecon/view

In [8]:
view data/ksp
bartview.py data/maps
view data/weights

We can see the dimensions using the `show` command:

In [9]:
bart show -m data/ksp

Type: complex float
Dimensions: 16
AoD:	1	68	180	20	1	1	1	1	1	1	18	1	1	1	1	1


The MRI-specific BART commands assume that the first three dimensions represent space/spatial k-space, the next two dimensions
represent coils and ESPIRiT maps, and the 10th dimension (zero-indexing) represents temporal phases.
We see the spatial dimensions are `1 X 68 x 180`, with 20 coils and 18 temporal phases.

To start, let's compute the sampling pattern and compare it to the k-space weighting mask.
The weighting mask assigns each phase encode a weight between zero and one, depending on the amount of
motion-corruption.

In [14]:
bart pattern data/ksp data/pat # generate a sampling pattern from k-space
bart join 1 data/pat data/weights data/pat_weights
bart show -m data/pat_weights

[1]+  Done                    view data/pat_weights_sum
Type: complex float
Dimensions: 16
AoD:	1	136	180	1	1	1	1	1	1	1	18	1	1	1	1	1


We can sum along the temporal dimension using the `fmac` (fused multiply accumulate) tool:

In [15]:
bart fmac -h

Usage: fmac [-C] [-s d] <input1> <input2> <output>

Multiply and accumulate.

-C		conjugate input2
-s b      	squash dimensions selected by bitmask b
-h		help


The bitmask is used to specify the dimensions we want to sum over. We can quickly compute a bitmask using the `bitmask`
tool:

In [16]:
bart bitmask 10 # bitmask for the temporal dimensions
bart bitmask 0 1 2 # bitmask for 3D spatial dimensions

1024
7


To sum over the temporal dimension, we create a BART file containing a single entry with value one, and sum over the
10th dimension:

In [17]:
bart ones 1 1 o
bart fmac -s $(bart bitmask 10) data/pat_weights o data/pat_weights_sum
view data/pat_weights_sum & # visualize the result

[1] 52214


We will use the `pics` tool to perform ESPIRiT-based parallel imaging and compressed sensing. The basic usage is

    bart pics [optimization options] [regularization options] kspace maps recon
    
See the full list with `bart pics -h` and `bart pics -Rh`.
There are several built-in regularization terms and transforms. We can see the options by invoking the help:

In [21]:
bart pics -h

Usage: pics [-l ...] [-r f] [-c] [-s f] [-i d] [-t <string>] [-n] [-g] [-p <string>] [-b d] [-e] [-W <string>] [-d d] [-u f] [-C d] [-f f] [-m ...] [-w f] [-S] <kspace> <sensitivities> <output>

Parallel-imaging compressed-sensing reconstruction.

-l1/-l2		toggle l1-wavelet or l2 regularization.
-r lambda      	regularization parameter
-c		real-value constraint
-s step      	iteration stepsize
-i iter      	max. number of iterations
-t file      	k-space trajectory
-n		disable random wavelet cycle spinning
-g		use GPU
-p file      	pattern or weights
-b blk      	Lowrank block size
-e		Scale stepsize based on max. eigenvalue
-W <img>      	Warm start with <img>
-d level      	Debug level
-u rho      	ADMM rho
-C iter      	ADMM max. CG iterations
-f rfov      	restrict FOV
-mSelect ADMM
-w val      	scaling
-S		Re-scale the image after reconstruction
-h		help


In [22]:
bart pics -Rh

Generalized regularization options (experimental)

-R <T>:A:B:C	<T> is regularization type (single letter),
		A is transform flags, B is joint threshold flags,
		and C is regularization value. Specify any number
		of regularization terms.

-R Q:C    	l2-norm in image domain
-R I:B:C  	l1-norm in image domain
-R W:A:B:C	l1-wavelet
-R T:A:B:C	total variation
-R T:7:0:.01	3D isotropic total variation with 0.01 regularization.
-R L:7:7:.02	Locally low rank with spatial decimation and 0.02 regularization.
-R M:7:7:.03	Multi-scale low rank with spatial decimation and 0.03 regularization.


## L1-wavelets in space and total variation in time.

Initialize the viewer with zero-filled reconstruction

In [25]:
bart pics -i 1 data/ksp data/maps recon_wavtv
view recon_wavtv&

[1]+  Done                    view recon_wavtv
[1] 52277


In [26]:
bart pics   -i 100 \
            -p data/weights \
            -R W:$(bart bitmask 0 1 2):0:0.001 \
            -R T:$(bart bitmask 10):0:.04 \
            data/ksp data/maps recon_wavtv


## Locally low rank (LLR).

Initialize the viewer with zero-filled reconstruction

In [28]:
bart pics -i 1 data/ksp data/maps recon_llr
view recon_llr&

[2] 52285


In [31]:
bart pics   -i 100 \
            -R L:$(bart bitmask 0 1 2):$(bart bitmask 0 1 2):0.01 \
            data/ksp data/maps recon_llr

[3]+  Done                    view recon_compare


Combine the reconstructions and compare

In [32]:
bart join 1 recon_wavtv recon_llr recon_compare # combine the reconstructions
view recon_compare &

[3] 52298
